# Experimenting to fix cracks & folds in basil  
Using the Optimizer to fix cracks & folds in basil

In [1]:
!mkdir -p ~/.cloudvolume/secrets
!cp /notebooks/*.json ~/.cloudvolume/secrets/
!pip install cloud-volume
!pip install scikit-image

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [42]:
from cloudvolume import CloudVolume
from cloudvolume.lib import Bbox, Vec
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn as nn
from optimize import Optimizer
from copy import deepcopy
from skimage.transform import resize
from scipy.ndimage.morphology import binary_erosion
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def use_src_mask_and_one_dst():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_offset = np.array([102716, 107077, 526])
    full_size = np.array([2048, 2048, 1])
    full_bbox = Bbox(full_offset, full_offset+full_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_bbox))
    dst_bbox = src_bbox + [0,0,1]
    mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_bbox))
    print(src_bbox)
    print(dst_bbox)
    print(mask_bbox)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_with_mask", mip=img_mip, 
                        info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    dst[dst_bbox.to_slices()] = src[dst_bbox.to_slices()]
    src_img = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    dst_img = src[dst_bbox.to_slices()][:,:,0,0] / 255.0

    fold = folds[mask_bbox.to_slices()][:,:,0,0] / 255.0
    crack = cracks[mask_bbox.to_slices()][:,:,0,0] / 255.0
    mask = resize(np.minimum(fold, crack), src_bbox.size3()[:2])
    field = flow.process(src_img, [dst_img], mask, [np.ones_like(dst_img)])
    pred = flow.render(src_img, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [4]:
def use_masks_and_two_dsts():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [5]:
def use_masks_and_four_dsts():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-2,-1,1,2]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_four_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [6]:
def use_filtered_mask_and_two_dsts():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_filtered_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask *= (src_mask < 1/255.0) | (src_mask > 10/255.0)
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [7]:
def use_filtered_mask_and_two_dsts2():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_filtered_mask_two_dsts2", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask *= (src_mask < 4/255.0) | (src_mask > 10/255.0)
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [8]:
def use_masks_and_two_dsts_mip1():
    img_mip = 1
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_mip1", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field
    

In [9]:
def use_masks_and_two_dsts_mip0():
    img_mip = 0
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102716, 107077, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_mip0", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field
    

In [10]:
def use_masks_and_two_dsts_crack():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    full_src_offset = np.array([102320, 111106, 526])
    full_src_size = np.array([2048, 2048, 1])
    full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
    src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
    src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
    dst_bboxes = []
    dst_mask_bboxes = []
    for z in [-1,1]:
        full_dst_bbox = full_src_bbox + [0,0,z]
        dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
        dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
        dst_bboxes.append(dst_bbox)
        dst_mask_bboxes.append(dst_mask_bbox)
        
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    
    dst_images = []
    dst_masks = []
    for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
        dst_slices = dst_bbox.to_slices()
        mask_slices = dst_mask_bbox.to_slices()
        dst[dst_slices] = src[dst_slices]
        
        dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
        dst_fold = folds[mask_slices][:,:,0,0] / 255.0
        dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
        dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
        dst_masks.append(dst_mask)
        
        
    src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
    src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
    src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
    
    src_mask = np.ones_like(src_mask) * ((src_mask < 6/255.0) | (src_mask > 20/255.0))
    
    field = flow.process(src_image, dst_images, src_mask, dst_masks)
    pred = flow.render(src_image, field)
    dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field

In [11]:
def use_gt_masks_and_two_dsts():
    mip = 2
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=mip, 
                                                                    parallel=2, fill_missing=True)
    mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v02_ground_truth_masks_two_dsts", 
          mip=mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102320, 111106, 526), (102716, 107077, 526)]
    full_src_size = np.array([2048, 2048, 1])
    for full_src_offset in offsets:
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox in dst_bboxes:
            dst_slices = dst_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_mask = (mask[dst_slices][:,:,0,0] == 0) / 1.0
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_mask = (mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)

In [12]:
def without_masks_and_two_dsts():
    mip = 2
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=mip, 
                                                                    parallel=2, fill_missing=True)
    mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v02_without_masks_two_dsts", 
          mip=mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102320, 111106, 526), (102716, 107077, 526)]
    full_src_size = np.array([2048, 2048, 1])
    for full_src_offset in offsets:
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox in dst_bboxes:
            dst_slices = dst_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_image = src[dst_slices][:,:,0,0] / 255.0
            dst_images.append(dst_image)
            # dst_mask = (mask[dst_slices][:,:,0,0] == 0) / 1.0
            dst_mask = np.ones_like(dst_image)
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        # src_mask = (mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0
        src_mask = np.ones_like(src_image)

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)

In [13]:
def use_masks_and_two_dsts_more_folds():
    img_mip = 2
    mask_mip = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(191721, 139068, 528),    # consecutive folds
               Vec(196097, 177069, 526),    # consecutive folds
               Vec(179301, 81627, 529),     # big crack
               Vec(167572, 73149, 522),     # track mark, no masks
               Vec(146350, 204341, 524),    # consecutive cracks
               Vec(130159, 162051, 525)     # consecutive cracks
              ]
    full_src_size = np.array([2048, 2048, 1])
    for full_src_offset in offsets:
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
            crack_max = np.max(dst_crack * (dst_crack < 20/255.0))
            crack_min = max(1, crack_max-3)
            dst_crack = np.ones_like(dst_crack) * ((dst_crack < crack_min) | (dst_crack > crack_max))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        crack_max = np.max(src_crack * (src_crack < 20/255.0))
        crack_min = max(1, crack_max-3)
        src_crack = np.ones_like(src_crack) * ((src_crack < crack_min) | (src_crack > crack_max))
        src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [14]:
def erode_mask(mask, radius=300, th=50):
    """Generate bitmask by thresholding mask below th, then eroding by radius
    """
    bitmask = mask < th
    f = np.ones((radius, radius))
    bitmask = binary_erosion(bitmask, structure=f, border_value=1)
    return ~bitmask*1

In [15]:
def use_masks_and_two_dsts_more_cracks():
    img_mip = 2
    mask_mip = 5
    radius = 250
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(179301, 81627, 529) - Vec(1024, 1024, 0),     # big crack
               Vec(146350, 204341, 525),                         # consecutive cracks
               Vec(130159, 162051, 525)                          # consecutive cracks
              ]
    sizes = [Vec(3072, 3072, 1),
             Vec(2048, 2048, 1),
             Vec(2048, 2048, 1)]
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [16]:
def use_masks_and_two_dsts_more_cracks_downsample6():
    img_mip = 2
    mask_mip = 5
    radius = 250
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_ndownsamples6", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=6, currn=6, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(179301, 81627, 529) - Vec(1024, 1024, 0),     # big crack
               Vec(146350, 204341, 525),                         # consecutive cracks
               Vec(130159, 162051, 525)                          # consecutive cracks
              ]
    sizes = [Vec(3072, 3072, 1),
             Vec(2048, 2048, 1),
             Vec(2048, 2048, 1)]
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])

        field = flow.process(src_image, dst_images, src_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [17]:
def make_field(dim, k=-0.005):
    field = torch.zeros((1,dim,dim,2))
    for i in range(dim):
        for j in range(dim):
            if i < dim - j:
                field[0,i,j,0] = 1*k
                field[0,i,j,1] = 1*k
            else:
                field[0,i,j,0] = -1*k
                field[0,i,j,1] = -1*k
    return field

In [18]:
def use_masks_and_two_dsts_big_crack_initialized_field():
    img_mip = 2
    mask_mip = 5
    radius = 250
    ndownsamples = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_initialized_field", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=ndownsamples, currn=ndownsamples, avgn=20, 
                             lambda1=0.9, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(179301, 81627, 529) - Vec(1024, 1024, 0)]
    sizes = [Vec(3072, 3072, 1)]
    field = 0
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        # src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        # src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        # src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
        src_crack = src_image < (210.0 / 255.0)
        src_mask = resize(src_crack, src_bbox.size3()[:2])

        dim = int(src_image.shape[-1] / (2**(ndownsamples - 1)))
        field = make_field(dim)
        # field = None
        field = flow.process(src_image, dst_images, src_mask, dst_masks, field=field)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field

In [19]:
def use_masks_and_two_dsts_big_crack_uninitialized_field():
    img_mip = 2
    mask_mip = 5
    radius = 250
    ndownsamples = 5
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)    
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/v01_mask_two_dsts_uninitialized_field", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=ndownsamples, currn=ndownsamples, avgn=20, 
                             lambda1=0.9, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(179301, 81627, 529) - Vec(1024, 1024, 0)]
    sizes = [Vec(3072, 3072, 1)]
    field = 0
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        # src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        # src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        # src_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
        src_crack = src_image < (210.0 / 255.0)
        src_mask = resize(src_crack, src_bbox.size3()[:2])

        # dim = int(src_image.shape[-1] / (2**(ndownsamples - 1)))
        # field = make_field(dim)
        field = None
        field = flow.process(src_image, dst_images, src_mask, dst_masks, field=field)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    return field

In [36]:
def smooth_loss_masks_and_two_dsts():
    img_mip = 2
    mask_mip = 5
    gt_mip = 2
    radius = 250
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)  
    loss_mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=gt_mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/smooth_loss_masks_and_two_dsts_v01", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102716, 107077, 526),     # test fold
               Vec(102320, 111106, 526),     # test crack
               Vec(102565, 103904, 527)      # no defect
              ]
    sizes = [Vec(2048, 2048, 1),
             Vec(2048, 2048, 1),
             Vec(2048, 2048, 1)]
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
            # dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        # src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        src_smooth_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
        src_loss_mask = (loss_mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0

        field = flow.process(src_image, dst_images, src_smooth_mask, src_loss_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [46]:
def smooth_loss_masks_and_two_dsts_mip3():
    img_mip = 3
    mask_mip = 5
    gt_mip = 2
    radius = 250
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)  
    loss_mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=gt_mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/smooth_loss_masks_and_two_dsts_mip3_v01", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102716, 107077, 526),     # test fold
               Vec(102320, 111106, 526),      # test crack
               Vec(102565, 103904, 527) 
              ]
    sizes = [Vec(2048, 2048, 1),
             Vec(2048, 2048, 1),
             Vec(2048, 2048, 1)]
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        # src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        src_smooth_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
        src_loss_mask = (loss_mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0

        field = flow.process(src_image, dst_images, src_smooth_mask, src_loss_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [25]:
def smooth_loss_masks_and_two_dsts_dilated_gt():
    img_mip = 2
    mask_mip = 5
    gt_mip = 2
    radius = 40
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)  
    loss_mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=gt_mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/smooth_loss_masks_and_two_dsts_dilated_gt_v01", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102716, 107077, 526),     # test fold
               Vec(102320, 111106, 526)      # test crack
              ]
    sizes = [Vec(2048, 2048, 1),
             Vec(2048, 2048, 1)]
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0]
            dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        # src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        # src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0]
        # src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        src_mask = loss_mask[src_bbox.to_slices()][:,:,0,0] == 0
        src_loss_mask = src_mask / 1.0
        f = np.ones((radius, radius))
        src_smooth_mask = binary_erosion(src_mask, structure=f, border_value=1)*1.0

        field = flow.process(src_image, dst_images, src_smooth_mask, src_loss_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [32]:
def smooth_loss_masks_and_two_dsts_nonzero_smooth_masks():
    img_mip = 2
    mask_mip = 5
    gt_mip = 2
    radius = 250
    eps = 50 / 255.0
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)  
    loss_mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=gt_mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/smooth_loss_masks_and_two_dsts_nonzero_smooth_masks_v01", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=5, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102716, 107077, 526),     # test fold
               Vec(102320, 111106, 526)      # test crack
              ]
    sizes = [Vec(2048, 2048, 1),
             Vec(2048, 2048, 1)]
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
            # dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0 + eps
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0 + eps
        # src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        src_smooth_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
        src_loss_mask = (loss_mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0

        field = flow.process(src_image, dst_images, src_smooth_mask, src_loss_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [38]:
def smooth_loss_masks_and_two_dsts_mip2_8():
    img_mip = 2
    mask_mip = 5
    gt_mip = 2
    radius = 250
    src = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/unmasked", mip=img_mip, 
                                                                    parallel=2, fill_missing=True)
    cracks = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/crack_detector_v3", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)
    folds = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/fold_detector_v1", mip=mask_mip,
                                                                     parallel=2, fill_missing=True)  
    loss_mask = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/mask/ground_truth", mip=gt_mip,
                                                                     parallel=2, fill_missing=True)
    dst = CloudVolume("gs://neuroglancer/basil_v0/son_of_alignment/v3.04/optimizer_tests/smooth_loss_masks_and_two_dsts_mip2_8_v01", 
          mip=img_mip, info=src.info, parallel=False, fill_missing=True, non_aligned_writes=True, cdn_cache=False)
    dst.commit_info();
    flow = Optimizer(ndownsamples=6, currn=5, avgn=20, lambda1=0.5, lr=0.1, eps=0.001, min_iter=100, max_iter=5000)
    offsets = [Vec(102716, 107077, 526),     # test fold
               Vec(102320, 111106, 526),     # test crack
               Vec(102565, 103904, 527)      # no defect
              ]
    sizes = [Vec(2048, 2048, 1),
             Vec(2048, 2048, 1),
             Vec(2048, 2048, 1)]
    for full_src_offset, full_src_size in zip(offsets, sizes):
        full_src_bbox = Bbox(full_src_offset, full_src_offset+full_src_size)
        src_bbox = Bbox.from_slices(src.slices_from_global_coords(full_src_bbox))
        src_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_src_bbox))
        dst_bboxes = []
        dst_mask_bboxes = []
        for z in [-1,1]:
            full_dst_bbox = full_src_bbox + [0,0,z]
            dst_bbox = Bbox.from_slices(src.slices_from_global_coords(full_dst_bbox))
            dst_mask_bbox = Bbox.from_slices(folds.slices_from_global_coords(full_dst_bbox))
            dst_bboxes.append(dst_bbox)
            dst_mask_bboxes.append(dst_mask_bbox)

        dst_images = []
        dst_masks = []
        for dst_bbox, dst_mask_bbox in zip(dst_bboxes, dst_mask_bboxes):
            dst_slices = dst_bbox.to_slices()
            mask_slices = dst_mask_bbox.to_slices()
            dst[dst_slices] = src[dst_slices]

            dst_images.append(src[dst_slices][:,:,0,0] / 255.0)
            dst_fold = folds[mask_slices][:,:,0,0] / 255.0
            dst_crack = cracks[mask_slices][:,:,0,0] / 255.0
            # dst_crack = erode_mask(dst_crack, radius=int(round(radius/2**mask_mip)))
            dst_mask = resize(np.minimum(dst_fold, dst_crack), dst_bbox.size3()[:2])
            dst_masks.append(dst_mask)


        src_image = src[src_bbox.to_slices()][:,:,0,0] / 255.0
        src_fold = folds[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        src_crack = cracks[src_mask_bbox.to_slices()][:,:,0,0] / 255.0
        # src_crack = erode_mask(src_crack, radius=int(round(radius/2**mask_mip)))
        src_smooth_mask = resize(np.minimum(src_fold, src_crack), src_bbox.size3()[:2])
        src_loss_mask = (loss_mask[src_bbox.to_slices()][:,:,0,0] == 0) / 1.0

        field = flow.process(src_image, dst_images, src_smooth_mask, src_loss_mask, dst_masks)
        pred = flow.render(src_image, field)
        dst[src_bbox.to_slices()] = (pred[0,0,:,:,np.newaxis, np.newaxis]*255).astype(np.uint8)
    

In [47]:
smooth_loss_masks_and_two_dsts_mip3()

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]
/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  8.55it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  7.28it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/1 [00:00<?, ?it/s]



(256, 256) 2 (256, 256) (256, 256) 2
Downsample factor 16x
[0.1775285]
[0.17503947]
[0.17173386]
[0.16875589]
[0.16605568]
[0.16366833]
[0.16131566]
[0.15902533]
[0.15693423]
[0.1550223]
[0.15308125]
[0.15124121]
[0.14942876]
[0.14781132]
[0.14619523]
[0.1446166]
[0.14304565]
[0.14164887]
[0.14025521]
[0.13885207]
[0.13748933]
[0.13627943]
[0.13504858]
[0.13380496]
[0.13260871]
[0.13153787]
[0.13044299]
[0.12933601]
[0.12831418]
[0.12730893]
[0.12633844]
[0.12536803]
[0.12447449]
[0.12356099]
[0.12265174]
[0.12180272]
[0.12104514]
[0.1201831]
[0.11938236]
[0.11860611]
[0.11791736]
[0.11715612]
[0.1164099]
[0.1157476]
[0.11508494]
[0.11439467]
[0.11373644]
[0.11313222]
[0.11254747]
[0.11189122]
[0.11129086]
[0.11077293]
[0.11019869]
[0.10963494]
[0.1090875]
[0.10858102]
[0.10806761]
[0.10753907]
[0.10710121]
[0.10657759]
[0.10609376]
[0.10564686]
[0.10523723]
[0.10476808]
[0.10429591]
[0.10388607]
[0.1035741]
[0.10321492]
[0.1028754]
[0.10258023]
[0.10229234]
[0.10196273]
[0.10166863]
[

[0.08169448]
[0.08166409]
[0.08169422]
[0.08167876]
[0.08167869]
[0.08166834]
[0.08167411]
[0.08164896]
[0.0816614]
[0.0816843]
[0.08166148]
[0.08165208]
[0.08164125]
[0.08166092]
[0.08163564]
[0.08167373]
[0.0816524]
[0.08161931]
[0.08163705]
[0.08167101]
[0.08163479]
[0.08159075]
[0.08166693]
[0.08164476]
[0.08162114]
[0.08162087]
[0.08160588]
[0.0816395]
[0.08162224]
[0.08162855]
[0.08160116]
[0.08160548]
[0.0816228]
[0.08161178]
[0.08159732]
[0.08159734]
[0.08162907]
[0.08160396]
[0.08158865]
[0.08158459]
[0.08159243]
[0.08161357]
[0.08160278]
[0.08156766]
[0.08157065]
[0.08160964]
[0.0815877]
[0.08155666]
[0.08159746]
[0.08159146]
[0.08157125]
[0.08155527]
[0.08158292]
[0.08157223]
[0.081554]
[0.0815963]
[0.08157428]
[0.08153357]
[0.08155756]
[0.08157481]
[0.08157134]
[0.08154653]
[0.08154804]
[0.08155806]
[0.08154841]
[0.08155408]
[0.08153524]
[0.08154114]
[0.08157174]
[0.0815421]
[0.08152442]
[0.0815241]
[0.08156886]
[0.08152947]
[0.0815315]
[0.08153625]
[0.08152863]
[0.08152784

[0.42059577]
[0.41891345]
[0.41904926]
[0.42058003]
[0.41849375]
[0.41872877]
[0.41787177]
[0.41743326]
[0.41912198]
[0.41956213]
[0.4194327]
[0.41982263]
[0.42006743]
[0.41856503]
[0.4185574]
[0.41789842]
[0.4162328]
[0.41620404]
[0.41432822]
[0.41379386]
[0.41595083]
[0.41586828]
[0.41402435]
[0.41447192]
[0.4127797]
[0.41212645]
[0.41445348]
[0.41323674]
[0.41223204]
[0.4127981]
[0.4132567]
[0.41400683]
[0.4132468]
[0.412705]
[0.41144162]
[0.41235226]
[0.41293427]
[0.41218612]
[0.41193527]
[0.410962]
[0.41192985]
[0.41420326]
[0.4126644]
[0.41271538]
[0.41276562]
[0.4114316]
[0.4118886]
[0.41220105]
[0.41074044]
[0.4092971]
[0.41100347]
[0.41204005]
[0.4107923]
[0.41067672]
[0.40951148]
[0.41027123]
[0.4113686]
[0.4105752]
[0.4093091]
[0.40898836]
[0.41028637]
[0.41077942]
[0.410424]
[0.41017595]
[0.40987507]
[0.4122231]
[0.4128583]
[0.41220954]
[0.41368562]
[0.41218656]
[0.41171157]
[0.41264054]
[0.41084787]
[0.40945005]
[0.40925714]
[0.41029713]
[0.4107309]
[0.41014564]
[0.4084964

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image: 100%|██████████| 1/1 [00:00<00:00,  9.74it/s]

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]


Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]



Downloading:   0%|          | 0/2 [00:00<?, ?it/s]

Downloa

...

(256, 256) 2 (256, 256) (256, 256) 2
Downsample factor 16x
[0.03017927]
[0.02951489]
[0.02899507]
[0.02860338]
[0.02830157]
[0.02802611]
[0.02779066]
[0.02757382]
[0.0273726]
[0.0271954]
[0.02701522]
[0.02685064]
[0.02670192]
[0.02654705]
[0.02641032]
[0.0262795]
[0.02614473]
[0.02603275]
[0.02591137]
[0.02579952]
[0.02570271]
[0.0255886]
[0.02549762]
[0.02540928]
[0.02530842]
[0.0252272]
[0.02515054]
[0.02506094]
[0.0249927]
[0.02491941]
[0.02484515]
[0.02478325]
[0.02471227]
[0.02465448]
[0.02459392]
[0.02453169]
[0.02448207]
[0.02442655]
[0.02436921]
[0.02432782]
[0.02427682]
[0.02422858]
[0.02418967]
[0.02413723]
[0.02410341]
[0.02406554]
[0.024014]
[0.02398555]
[0.02395253]
[0.02390596]
[0.02388024]
[0.02384918]
[0.02380819]
[0.02378375]
[0.02374968]
[0.02371675]
[0.02369026]
[0.02365902]
[0.02363442]
[0.02360675]
[0.0235751]
[0.02355548]
[0.02352847]
[0.02349889]
[0.02348582]
[0.0234548]
[0.02343203]
[0.02342002]
[0.02338719]
[0.02336736]
[0.02335624]
[0.02332656]
[0.02330869]
[0

[0.06489668]
[0.06487759]
[0.06486416]
[0.06486592]
[0.06482214]
[0.06484606]
[0.06482099]
[0.06479002]
[0.06480946]
[0.06478134]
[0.06475791]
[0.06477527]
[0.06474704]
[0.06473602]
[0.06474204]
[0.06471071]
[0.06470782]
[0.06470612]
[0.06468402]
[0.06469426]
[0.06466155]
[0.06467861]
[0.0646426]
[0.06464265]
[0.06465061]
[0.06460953]
[0.06464189]
[0.064614]
[0.06458072]
[0.06463087]
[0.06458479]
[0.0645749]
[0.0645835]
[0.06457271]
[0.06456426]
[0.064572]
[0.06453357]
[0.06455356]
[0.06453937]
[0.06452778]
[0.06453352]
[0.06452684]
[0.06452238]
[0.06450491]
[0.06449373]
[0.06451939]
[0.0644805]
[0.06450257]
[0.06448703]
[0.0644705]
[0.06449144]
[0.06446265]
[0.06445692]
[0.06447404]
[0.0644578]
[0.06444122]
[0.06446616]
[0.06443542]
[0.06444285]
[0.0644447]
[0.06440899]
[0.06443977]
[0.06443106]
[0.06440765]
[0.06443481]
[0.06439008]
[0.06442894]
[0.06438881]
[0.06440128]
[0.06441309]
[0.06437995]
[0.06440394]
[0.06438598]
[0.06437933]
[0.06438506]
[0.06436443]
[0.06438447]
[0.0643726

Rechunking image: 0it [00:00, ?it/s]
Uploading: 0it [00:00, ?it/s]
Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

Rechunking image:   0%|          | 0/1 [00:00<?, ?it/s]

[0.7392118]
[0.73894596]
[0.739449]
done: 1378
torch.Size([1, 256, 256, 2])



Rechunking image: 100%|██████████| 1/1 [00:00<00:00, 10.11it/s]
